Give xgb params dict and feature params dict,

Make xgb.cv rmse, on-test score, in-train score, retrain-on-all

In [1]:
import talib as ta
import numpy as np
import pandas as pd
import gc
import time
import xgboost as xgb
import os
from random import sample
from os.path import exists
import json
import pickle

import matplotlib.pylab as plt
%matplotlib inline

In [2]:
from local_tune_method import params_xgb, params_train,make_tune

params_version =1
os.system(f"mkdir model_local_{params_version}")

save_folder = f"model_local_{params_version}"
with open(save_folder+ f'/README{params_version}.txt', 'w') as f:
    f.write(f'Version {params_version}\n')

## data

In [3]:
df_train=pd.read_feather('./data/new_data.ftr',
                        columns=['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close',
                               'Volume', 'Target', 'Weight', 'lr_15', 'Mkt_lrt_15','Crypto_Index'])

print('finished loading')
print(df_train.columns)

finished loading
Index(['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close',
       'Volume', 'Target', 'Weight', 'lr_15', 'Mkt_lrt_15', 'Crypto_Index'],
      dtype='object')


In [4]:
df_test = df_train[df_train['timestamp']>=df_train['timestamp'].quantile(0.95)]
df_retrain = df_train[(df_train['timestamp']>df_train['timestamp'].quantile(0.45)) & \
                      (df_train['timestamp']<df_train['timestamp'].quantile(0.95))]

del df_train
gc.collect()

0

## Parameter candidates

In [5]:
list_xgb_params=\
[
{'subsample': 0.7,
 'reg_lambda': 2.0,
 'reg_alpha': 1.0,
 'min_child_weight': 1.1,
 'max_depth': 5,
 'learning_rate': 0.41,
 'colsample_bytree': 0.6},
{'subsample': 0.7,
 'reg_lambda': 1.0,
 'reg_alpha': 0.0,
 'min_child_weight': 1.1,
 'max_depth': 5,
 'learning_rate': 0.41,
 'colsample_bytree': 0.6}
 ]

list_feature_params=\
    [
    {'std_lr_15': 30, 'std_Mkt_lrt_15': 10, 'std_Crypto_Index': 30, 
        'rsi': 30, 'adx': 50,
        'macd_sig': 15, 'macd_s': 10, 'macd_l': 60, 
        'lrtn': 50, 'fastk2': 10, 'fastk1': 15, 
        'beta_s': '6h', 'beta_l': '2d', 
         'vol_sum': 15},
    {'std_lr_15': 30, 'std_Mkt_lrt_15': 10, 'std_Crypto_Index': 30, 
        'rsi': 30,'adx': 50,
        'macd_sig': 15, 'macd_s': 10, 'macd_l': 60, 
        'lrtn': 50, 'fastk2': 15, 'fastk1': 10, 
        'beta_s': '6h', 'beta_l': '2d', 
         'vol_sum': 15},
    {'std_lr_15': 30, 'std_Mkt_lrt_15': 10, 'std_Crypto_Index': 30, 
        'rsi': 30,'adx': 60, 
        'macd_sig': 15, 'macd_s': 10, 'macd_l': 60, 
        'lrtn': 50, 'fastk2': 10, 'fastk1': 15, 
        'beta_s': '6h', 'beta_l': '2d', 
         'vol_sum': 15}
]

## Start tune

In [6]:
params_xgb.update(list_xgb_params[0])
feature_params = list_feature_params[0]

score_cv,score_on_test,score_on_train = make_tune(df_retrain,df_test,params_xgb, feature_params, save_folder)

cv test-rmse: 0.0019914
Start train model on the sub-sample train set for scoring(9mins)
score-on-test: 0.01199323726604295


In [ ]:
with open(save_folder+ f'/README{params_version}.txt', 'a') as f:
    notes = f"""
    Params setting: {params_xgb}
    Feature setting: {feature_params}
    cv score: {score_cv}
    score_on_test: {score_on_test}
    score_on_train: {score_on_train}
    """
    f.write(f'{notes}\n')